In [ ]:
import pandas as pd
import numpy as np
import psycopg2

pd.set_option('display.max_columns', None)

In [ ]:
conn = psycopg2.connect(
        host="localhost",
        database="bloodmoneydb",
        user="postgres",
        password="password"
)

In [ ]:
sql_query = """
    SELECT *
    FROM model_input_tbl_raw
"""

df = pd.read_sql_query(sql_query, con=conn)

In [ ]:
keepers = [
    'index', 
    'red_color',
    'red_date',
    'red_fighter_id',
    'red_record_tot_fights',
    'red_record_win',
    'red_record_loss',
    'red_height',
    'red_weight',
    'red_stance',
    'red_outcome',
    'red_accolade_name',
    'red_weight_class_id',
    'red_weight_class_name',
    'red_possible_rds',
    'red_method',
    'red_ending_round_num',
    'red_age',
    'red_is_local',
    'red_is_national',
    'red_cumulative_possible_rds',
    'red_cumulative_fought_rds',
    'red_pct_rds_fought',
    'red_win_streak',
    'blue_color',
    'blue_date',
    'blue_fighter_id',
    'blue_record_tot_fights',
    'blue_record_win',
    'blue_record_loss',
    'blue_height',
    'blue_weight',
    'blue_stance',
    'blue_outcome',
    'blue_accolade_name',
    'blue_weight_class_id',
    'blue_weight_class_name',
    'blue_possible_rds',
    'blue_method',
    'blue_ending_round_num',
    'blue_age',
    'blue_is_local',
    'blue_is_national',
    'blue_cumulative_possible_rds',
    'blue_cumulative_fought_rds',
    'blue_pct_rds_fought',
    'blue_win_streak'
]

In [ ]:
df = df[keepers].copy()

To Do:

* Get counts of the weight class ids and the weight class names to see if there are any anomalies 

* Get the distribution of wins by color type, needs to be approximately 50%

* Create variable for every red-blue combo of locality and stance

In [ ]:
df.groupby(['red_weight_class_id', 'red_weight_class_name']).size()

In [ ]:
df.groupby(['red_color', 'red_outcome']).size()